In [11]:
import requests
import pandas as pd
import io, s3fs, json

# COI  
Access Child Opportunity Index (COI) raw indicator values from https://data.diversitydatakids.org/dataset/coi20-child-opportunity-index-2-0-database/resource/f16fff12-b1e5-4f60-85d3-3a0ededa30a0.

In [12]:
r = requests.post('https://data.diversitydatakids.org/files/COI/COI20/raw/raw.csv')
coi_rawindicator=pd.read_csv(io.StringIO(r.text), sep=',')
coi_rawindicator.head()

,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,ED_APENR,...,HE_SUPRFND,HE_RSEI,SE_POVRATE,SE_PUBLIC,SE_HOME,SE_OCC,SE_MHE,SE_EMPRAT,SE_JOBPROX,SE_SINGLE
0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0.349463,...,-13.815511,5.933720,9.240363,14.398734,84.335442,27.972027,60321.0,68.187744,5.224787,17.256638
1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,0.252991,...,-13.815511,7.015026,10.677506,12.068965,75.198936,38.479000,67826.0,82.987556,1.052632,26.556019
2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0.350239,...,-13.815511,6.083863,10.511090,13.028169,58.568073,40.000000,44039.0,59.521217,1.727862,48.818897
3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,0.253207,...,-13.815511,7.306938,22.413794,24.137930,61.941254,30.516430,41287.0,60.339661,0.931315,47.470818
4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,0.349768,...,-13.815511,6.162114,11.823454,9.035088,76.315788,31.036932,46002.0,82.875099,1.788375,31.969309


Save raw data to S3.

In [13]:
with open('/Volumes/Key/aws_keys.json') as f:
  creds = json.load(f)

In [14]:
def s3upload(name,df):
    x = df.to_csv(None).encode()
    fs = s3fs.S3FileSystem(key=creds['AWSAccessKeyId'], secret=creds['AWSSecretKey'])
    with fs.open('s3://bleeding-hearts/rawdata/' + name + '.csv', 'wb') as f:
        f.write(x)
    print('saved')

In [15]:
s3upload('coi_rawindicator',coi_rawindicator)

saved


Test opening data from S3 bucket.

In [16]:
test = pd.read_csv('s3://bleeding-hearts/rawdata/coi.csv')

In [17]:
test.head()

,Unnamed: 0,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,...,HE_SUPRFND,HE_RSEI,SE_POVRATE,SE_PUBLIC,SE_HOME,SE_OCC,SE_MHE,SE_EMPRAT,SE_JOBPROX,SE_SINGLE
0,0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,...,-13.815511,5.933720,9.240363,14.398734,84.335442,27.972027,60321.0,68.187744,5.224787,17.256638
1,1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,...,-13.815511,7.015026,10.677506,12.068965,75.198936,38.479000,67826.0,82.987556,1.052632,26.556019
2,2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,...,-13.815511,6.083863,10.511090,13.028169,58.568073,40.000000,44039.0,59.521217,1.727862,48.818897
3,3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,...,-13.815511,7.306938,22.413794,24.137930,61.941254,30.516430,41287.0,60.339661,0.931315,47.470818
4,4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,...,-13.815511,6.162114,11.823454,9.035088,76.315788,31.036932,46002.0,82.875099,1.788375,31.969309


In [18]:
r = requests.post('https://data.diversitydatakids.org/files/COI/COI20/index/index.csv')
coi_index=pd.read_csv(io.StringIO(r.text), sep=',')
coi_index.head()

,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,z_ED_nat,...,r_SE_stt,r_COI_stt,c5_ED_met,c5_HE_met,c5_SE_met,c5_COI_met,r_ED_met,r_HE_met,r_SE_met,r_COI_met
0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0.001198,...,67,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,0.008932,...,83,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0.013306,...,31,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,-0.003518,...,26,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,-0.014533,...,77,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
s3upload('coi_index',coi_index)

saved


In [20]:
r = requests.post('https://data.diversitydatakids.org/files/COI/COI20/pop/pop.csv')
coi_pop=pd.read_csv(io.StringIO(r.text), sep=',')
coi_pop.head()

,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,aian,api,black,hisp,other2,nonwhite,white,total
0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0,0,56,0,0,56,368,424
1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,0,14,19,12,20,65,349,414
2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0,33,229,0,24,286,211,497
3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,0,10,229,0,18,257,251,508
4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,0,15,194,26,0,235,643,878


In [21]:
s3upload('coi_pop',coi_pop)

saved


In [22]:
r = requests.post('https://data.diversitydatakids.org/files/COI/COI20/zscores/zscores.csv')
coi_z=pd.read_csv(io.StringIO(r.text), sep=',')
coi_z.head()

,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,zED_APENR,...,zHE_RSEI,zSE_POVRATE,zSE_PUBLIC,zSE_HOME,zSE_OCC,zSE_MHE,zSE_EMPRAT,zSE_JOBPROX,zSE_SINGLE,zSE_ECRES
0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0.058250,...,0.217542,0.515539,-0.112855,0.823233,-0.391742,-0.084708,-0.689880,0.459048,0.936724,0.157800
1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,-0.372527,...,-0.164622,0.393041,0.096836,0.398006,0.334608,0.169833,0.829066,1.065432,0.426814,0.321208
2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0.061718,...,0.164477,0.407226,0.010503,-0.376019,0.439755,-0.636931,-1.579350,0.967294,-0.793917,-0.027652
3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,-0.371563,...,-0.267792,-0.607322,-0.989432,-0.219026,-0.215846,-0.730268,-1.495351,1.083064,-0.719998,-0.672206
4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,0.059614,...,0.136821,0.295364,0.369901,0.449986,-0.179864,-0.570353,0.817524,0.958499,0.129990,0.074177


In [23]:
s3upload('coi_z',coi_z)

saved


# ACS  
Load the ACS 1-year DP03 table for 2018.  
Using Data Profiles from https://www.census.gov/data/developers/data-sets/acs-1year.2018.html.  

get=group(DP03) based on https://api.census.gov/data/2018/acs/acs1/profile/variables.html  
for=metropolitan%20statistical%20area/micropolitan%20statistical%20area: based on https://api.census.gov/data/2018/acs/acs1/examples.html

In [14]:
with open('/Volumes/Key/census_key.txt') as key:
    api_key=key.read().strip()

In [15]:
url=f'https://api.census.gov/data/2018/acs/acs1/profile?get=group(DP03)&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key={api_key}'

In [16]:
r = requests.get(url)

In [17]:
df = pd.DataFrame(columns=r.json()[0],data=r.json()[1:])
df

,GEO_ID,DP03_0001E,DP03_0001M,DP03_0001PE,DP03_0001PM,DP03_0002E,DP03_0002M,DP03_0002PE,DP03_0002PM,DP03_0003E,...,DP03_0135PMA,DP03_0136EA,DP03_0136MA,DP03_0136PEA,DP03_0136PMA,DP03_0137EA,DP03_0137MA,DP03_0137PEA,DP03_0137PMA,metropolitan statistical area/micropolitan statistical area
0,310M400US10300,79720,460,79720,-888888888,46381,1607,58.2,2.1,46330,...,None,(X),(X),None,None,(X),(X),None,None,10300
1,310M400US10460,53166,1077,53166,-888888888,26839,2262,50.5,4.0,24507,...,None,(X),(X),None,None,(X),(X),None,None,10460
2,310M400US10780,120480,655,120480,-888888888,66089,3270,54.9,2.7,65480,...,None,(X),(X),None,None,(X),(X),None,None,10780
3,310M400US12060,4676101,5398,4676101,-888888888,3139158,18687,67.1,0.4,3134908,...,None,(X),(X),None,None,(X),(X),None,None,12060
4,310M400US12580,2257415,2804,2257415,-888888888,1504130,10563,66.6,0.5,1483337,...,None,(X),(X),None,None,(X),(X),None,None,12580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,310M400US46700,357799,938,357799,-888888888,229781,4210,64.2,1.2,226696,...,None,(X),(X),None,None,(X),(X),None,None,46700
515,310M400US47700,62008,681,62008,-888888888,43397,1726,70.0,2.7,43397,...,None,(X),(X),None,None,(X),(X),None,None,47700
516,310M400US48580,84938,452,84938,-888888888,56974,1566,67.1,1.9,56941,...,None,(X),(X),None,None,(X),(X),None,None,48580
517,310M400US49340,775749,1555,775749,-888888888,511020,6624,65.9,0.9,510306,...,None,(X),(X),None,None,(X),(X),None,None,49340


Upload raw ACS data to S3

In [18]:
s3upload('acs',df)

saved


Getting descriptions for the headers from https://api.census.gov/data/2018/acs/acs1/profile/variables.json  
Converting to dataframe

In [19]:
headers=requests.get('https://api.census.gov/data/2018/acs/acs1/profile/variables.json')

In [20]:
d=headers.json()
d.keys()

dict_keys(['variables'])

In [21]:
d['variables'].keys()

dict_keys(['for', 'in', 'ucgid', 'DP02_0126E', 'DP05_0050PE', 'DP04_0047E', 'DP02PR_0083PE', 'DP02PR_0106PE', 'DP04_0046E', 'DP04_0016PE', 'DP02_0125E', 'DP02_0054PE', 'DP04_0049E', 'DP03_0048PE', 'DP04_0029PE', 'DP02_0124E', 'DP04_0048E', 'PUMA5', 'DP02_0123E', 'DP02PR_0017PE', 'DP04_0043E', 'DP02PR_0128E', 'DP03_0022PE', 'DP02_0122E', 'DP04_0003PE', 'DP04_0042E', 'DP02_0143PE', 'DP02PR_0127E', 'DP02_0010PE', 'DP02_0121E', 'DP02PR_0070PE', 'DP02_0130PE', 'DP03_0111PE', 'DP04_0045E', 'DP04_0136PE', 'DP02PR_0137PE', 'DP02_0120E', 'DP03_0109E', 'DP02_0067PE', 'DP04_0044E', 'DP02PR_0129E', 'DP03_0035PE', 'DP04_0110PE', 'DP02PR_0083E', 'DP05_0028E', 'DP05_0028PE', 'DP02_0085PE', 'DP03_0066PE', 'DP04_0047PE', 'DP03_0107E', 'DP05_0081PE', 'DP02PR_0082E', 'DP02PR_0065PE', 'DP05_0029E', 'DP03_0108E', 'DP02PR_0052PE', 'DP02PR_0085E', 'DP03_0105E', 'DP04_0123PE', 'DP02PR_0084E', 'DP02_0072PE', 'DP02_0019PE', 'DP03_0106E', 'DP03_0079PE', 'DP03_0103E', 'DP05_0024E', 'DP02_0041PE', 'DP02PR_0119PE',

In [22]:
df_headers=pd.DataFrame.from_dict(d['variables'])
df_headers

,for,in,ucgid,DP02_0126E,DP05_0050PE,DP04_0047E,DP02PR_0083PE,DP02PR_0106PE,DP04_0046E,DP04_0016PE,...,DP02PR_0110E,DP03_0115PE,GEOCOMP,DP02_0099E,DP04_0096E,DP02PR_0001PE,DP03_0039PE,DP02_0098E,DP04_0095PE,DP02_0036PE
label,Census API FIPS 'for' clause,Census API FIPS 'in' clause,Uniform Census Geography Identifier clause,Estimate!!ANCESTRY!!Total population!!Danish,Percent Estimate!!RACE!!Total population!!One ...,Estimate!!HOUSING TENURE!!Occupied housing uni...,Percent Estimate!!RESIDENCE 1 YEAR AGO!!Popula...,Percent Estimate!!WORLD REGION OF BIRTH OF FOR...,Estimate!!HOUSING TENURE!!Occupied housing uni...,Percent Estimate!!YEAR STRUCTURE BUILT!!Total ...,...,Estimate!!LANGUAGE SPOKEN AT HOME!!Population ...,Percent Estimate!!HEALTH INSURANCE COVERAGE!!C...,GEO_ID Component,Estimate!!YEAR OF ENTRY!!Population born outsi...,Estimate!!SELECTED MONTHLY OWNER COSTS (SMOC)!...,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total ho...,Percent Estimate!!INDUSTRY!!Civilian employed ...,Estimate!!YEAR OF ENTRY!!Population born outsi...,Percent Estimate!!SELECTED MONTHLY OWNER COSTS...,Percent Estimate!!FERTILITY!!Number of women 1...
concept,Census API Geography Specification,Census API Geography Specification,Census API Geography Specification,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,ACS DEMOGRAPHIC AND HOUSING ESTIMATES,SELECTED HOUSING CHARACTERISTICS,SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO,SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO,SELECTED HOUSING CHARACTERISTICS,SELECTED HOUSING CHARACTERISTICS,...,SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO,SELECTED ECONOMIC CHARACTERISTICS,NaN,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,SELECTED HOUSING CHARACTERISTICS,SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO,SELECTED ECONOMIC CHARACTERISTICS,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,SELECTED HOUSING CHARACTERISTICS,SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...
predicateType,fips-for,fips-in,ucgid,int,float,int,float,float,int,int,...,int,float,string,int,int,int,float,int,float,int
group,N/A,N/A,N/A,DP02,DP05,DP04,DP02PR,DP02PR,DP04,DP04,...,DP02PR,DP03,N/A,DP02,DP04,DP02PR,DP03,DP02,DP04,DP02
limit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
predicateOnly,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attributes,NaN,NaN,NaN,"DP02_0126EA,DP02_0126M,DP02_0126MA","DP05_0050PEA,DP05_0050PM,DP05_0050PMA","DP04_0047EA,DP04_0047M,DP04_0047MA","DP02PR_0083PEA,DP02PR_0083PM,DP02PR_0083PMA","DP02PR_0106PEA,DP02PR_0106PM,DP02PR_0106PMA","DP04_0046EA,DP04_0046M,DP04_0046MA","DP04_0016PEA,DP04_0016PM,DP04_0016PMA",...,"DP02PR_0110EA,DP02PR_0110M,DP02PR_0110MA","DP03_0115PEA,DP03_0115PM,DP03_0115PMA",NaN,"DP02_0099EA,DP02_0099M,DP02_0099MA","DP04_0096EA,DP04_0096M,DP04_0096MA","DP02PR_0001PEA,DP02PR_0001PM,DP02PR_0001PMA","DP03_0039PEA,DP03_0039PM,DP03_0039PMA","DP02_0098EA,DP02_0098M,DP02_0098MA","DP04_0095PEA,DP04_0095PM,DP04_0095PMA","DP02_0036PEA,DP02_0036PM,DP02_0036PMA"
required,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,default displayed,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Upload header dataframe to S3

In [25]:
s3upload('acs_headers',df_headers)

saved
